In [1]:
# from readability import Document
# import requests

# url = 'https://money.cnn.com/2018/08/12/investing/turkish-lira-currency-crisis/index.html'
# response = requests.get(url)
# doc = Document(remove_sometag(response.text))

# tanslate and index2examp

In [2]:
import spacy
from sacremoses import MosesDetokenizer
from googletrans import Translator

In [3]:
file = open('../../sentParsed/parsed_sent(coca).txt', 'r')
sentences = []
for line in file:
    index, line = line.split(' ||| ')
    sentences.append(line)
file.close()

In [4]:
file = open('../preprocess/Camb/index2exam.txt', 'r')
camIndex2Ex = {}
for line in file:
#     print(line.strip().split(' ||| '))

    index, examp = line.strip().split(' ||| ')
    index = int(index)
    
    if len(examp.split('\t')) == 3:
        en, ch, phrase = examp.split('\t')
        camIndex2Ex[index] = [en, ch, phrase]
    else: 
        try:
            en, ch = examp.split('\t')
        except:
            en = examp.strip('\t')
            ch = ''
        camIndex2Ex[index] = [en, ch]
    

In [5]:
# 保留住翻譯
matching = dict()

file = open('GPs.txt', 'r')
GP = eval(file.read())
file.close()

file = open('phrase.txt', 'r')
phrase = eval(file.read())
file.close()

for pos in GP.keys():
    for word in GP[pos].keys():
        for pat, colls, examp in GP[pos][word]:
            en, ch, source = examp
            matching[en] = ch
            
for head in phrase.keys():
    for p in phrase[head].keys():
        for pat, colls, examp in phrase[head][p]:
            en, ch, source = examp
            matching[en] = ch

In [6]:
from collections import defaultdict

GPs = defaultdict(lambda: defaultdict(lambda: []))
phrase = defaultdict(lambda: defaultdict(lambda: []))

In [7]:
# def translate(enExamp):
#     translator = Translator()
#     chExamp = translator.translate(enExamp, dest='zh-TW').text
#     return enExamp, chExamp

import requests
import json
import xml.etree.ElementTree as ET

def translate(en):
    # https://hk.saowen.com/a/fa191e99ed7c015c5342f126305ee324ff6bcee1779d8135643eed7df4f94d98 各項網站翻譯！！！
    # google 'http://translate.google.cn/translate_a/single?client=gtx&dt=t&dj=1&ie=UTF-8&sl=auto&tl=zh_TW&q='+en
    #        result['sentences'][0]['orig'], result['sentences'][0]['trans']
    # Bing url = 'http://api.microsofttranslator.com/v2/Http.svc/Translate?appId=AFC76A66CF4F434ED080D245C30CF1E71C22959C&from=&to=zh-tw&text='+en
    #        tree = ET.ElementTree(ET.fromstring(response.text))
    #        return en, tree.getroot().text
    url = 'http://fanyi.baidu.com/transapi?from=auto&to=cht&query='+en
    response = requests.get(url)
    
    result = json.loads(response.text)
    return result['data'][0]['src'], result['data'][0]['dst']

def getDependencies(line):
    dependency = line[:-1].split(', ')
    dependency = [ word.split('\t') for word in dependency ]
    dependency = [ (int(iword), word, lemma, dep, tag, pos, int(ihead), NER) for iword, word, lemma, dep, tag, pos, ihead, NER in dependency]
    return dependency
    
detokenizer = MosesDetokenizer()
def deTokenize(tokens):
    return detokenizer.detokenize(tokens, return_str=True)

In [8]:
translate('Can school children get excited about opera?')

('Can school children get excited about opera?', '學童能對歌劇感到興奮嗎？')

In [9]:

def jobs(line):
    key, value = line.strip('\n').split('\t')
    word, pos, pat = key.split(', ')
    colls , index, source = value.split('|||')
    colls = eval(colls)
    index = int(index)
    en = ''; ch = ''
    
    if source == 'cam':
        phraseSource = ''
        if len(camIndex2Ex[index]) == 3:
            en, ch, phraseSource = camIndex2Ex[index]
        else:
            en, ch = camIndex2Ex[index]
        hasChinese = ch != ''
        if not hasChinese:
            if en in matching: ch = matching[en]
            else: 
                en, ch = translate(en)
                if not ch: print(en, ch)
                    
        if pos not in {'V', 'N', 'ADJ'}:
            # word -> word, pos -> phrase
            if phraseSource: phrase[word][pos] += [(pat, colls, (en, ch, phraseSource))]
            elif hasChinese: phrase[word][pos] += [(pat, colls, (en, ch, 'cam-zht'))]
            else: phrase[word][pos] += [(pat, colls, (en, ch, 'cam-en'))]
        else:
            if phraseSource: GPs[pos][word] += [(pat, colls, (en, ch, phraseSource))]
            elif hasChinese: GPs[pos][word] += [(pat, colls, (en, ch, 'cam-zht'))]
            else: GPs[pos][word] += [(pat, colls, (en, ch, 'cam-en'))]
    else:
        if index == -1:
            if pos not in {'V', 'N', 'ADJ'}:
                phrase[word][pos] += [(pat, colls, (en, ch, 'coca'))]
            else:
                GPs[pos][word] += [(pat, colls, (en, ch, 'coca'))]
        else:
            dependency = getDependencies(sentences[index])
            en = deTokenize([ x[1] for x in dependency ])

            if en in matching: ch = matching[en]
            else: 
                en, ch = translate(en)
                if not ch: print(en, ch)
    
            if pos not in {'V', 'N', 'ADJ'}:
                phrase[word][pos] += [(pat, colls, (en, ch, 'coca'))]
            else:
                GPs[pos][word] += [(pat, colls, (en, ch, 'coca'))]
            

In [11]:
from joblib import Parallel, delayed

Parallel(n_jobs=20, verbose=2, require='sharedmem')(delayed(jobs)(line) for line in open('output.txt', 'r'))


[Parallel(n_jobs=20)]: Using backend ThreadingBackend with 20 concurrent workers.
[Parallel(n_jobs=20)]: Done   1 tasks      | elapsed:    0.0s
[Parallel(n_jobs=20)]: Done 122 tasks      | elapsed:    0.1s
[Parallel(n_jobs=20)]: Done 325 tasks      | elapsed:    1.0s
[Parallel(n_jobs=20)]: Done 608 tasks      | elapsed:    4.9s
[Parallel(n_jobs=20)]: Done 973 tasks      | elapsed:    7.2s
[Parallel(n_jobs=20)]: Done 1418 tasks      | elapsed:    9.0s
[Parallel(n_jobs=20)]: Done 1945 tasks      | elapsed:   13.4s
[Parallel(n_jobs=20)]: Done 2552 tasks      | elapsed:   15.4s
[Parallel(n_jobs=20)]: Done 3241 tasks      | elapsed:   21.4s
[Parallel(n_jobs=20)]: Done 4010 tasks      | elapsed:   24.6s
[Parallel(n_jobs=20)]: Done 4861 tasks      | elapsed:   28.8s
[Parallel(n_jobs=20)]: Done 5792 tasks      | elapsed:   34.2s
[Parallel(n_jobs=20)]: Done 6805 tasks      | elapsed:   38.9s
[Parallel(n_jobs=20)]: Done 7898 tasks      | elapsed:   45.5s
[Parallel(n_jobs=20)]: Done 9073 tasks   

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [12]:
# 由於joblib無法保證大小順序
for pos in GP.keys():
    for word in GP[pos].keys():
        GPs[pos][word].sort(key=lambda x: -int(x[0].split('%')[1]))
            
for head in phrase.keys():
    for p in phrase[head].keys():
        phrase[head][p].sort(key=lambda x: -int(x[0].split('%')[1]))

In [15]:
GPs['V']['compel']
# for pos in GP.keys():
#     for word in GP[pos].keys():
#         for pat in GPs[pos][word]:
#             if 'V adv' in pat[0]:
#                 print(word, pat)

[('V n%38',
  ['testimony', 'compliance', 'conclusion'],
  ('The new circumstances compelled a change in policy.',
   '新情況迫使在政策上作出改變。',
   'cam-zht')),
 ('V to inf%14',
  ['return', 'defend', 'make'],
  ('He hated confessions, and Bertie seemed compelled to make one.',
   '他憎恨懺悔，Bertie似乎被迫做出懺悔。',
   'coca')),
 ('V n to n%8',
  ['reader_communication', 'people_part', '&lt;GPE&gt;_condemnation'],
  ("It did n't take much to compel people to part with their dough.",
   '它並沒有花太多的力氣強迫人們分開麵團。',
   'coca')),
 ('V adv%7', ['apparently', 'probably', 'effectively'], ('', '', 'coca')),
 ('V by n%7',
  ['need', 'goal', 'job'],
  ('I seek a sense of missionaries as constrained and compelled by a comparable goal.',
   '我尋求傳教士的感覺，被一個可比的目標所約束和強迫。',
   'coca'))]

In [14]:
import json

file = open('GPs.txt', 'w')
file.write(str(json.loads(json.dumps(GPs))))
file.close()

file = open('phrase.txt', 'w')
file.write(str(json.loads(json.dumps(phrase))))
file.close()

In [ ]:
# p_ = set()
# word_ = set()

# for pos in GPs.keys():
#     for word in GPs[pos].keys():
#         for pat, colls, examp in GPs[pos][word]:
#             if pat.startswith('be V-ed'): 
#                 p_.add(pat.split('%')[0])
#                 word_.add(word)
                
# print(word_)